In [1]:
import requests
import json
import pandas as pd
from flatten_json import flatten
from utils.io import dict_to_yaml
id='lgulden'
apikey='4d1a4bc3be920e859b3862a25d3725d741028d42'

query='https://lda.senate.gov/api/v1/filings/?client_name=denbury&filing_year=2023'

In [379]:
def parse_dollars_spent(income,expense):
    if (income is None) & (expense is None):
        return 'income and expense are none',0.0
    if (income is None):
        return 'corporation lobbying for itself',float(expense)
    if (expense is None):
        return 'hired lobbying firm',float(income)
    else:
        return 'both income and expense > $0',float(income)+float(expense)
    
def initialize_row(entity_df,result,filing_id):
    #set up row dictionary using entity booleans
    initialize_row_dict=dict(zip([x.lower() for x in list(entity_df['name'])],[0]*len(list(entity_df['name']))))
    initialize_row_dict['who is lobbying'],initialize_row_dict['dollars_spent_lobbying']=parse_dollars_spent(result['income'],result['expenses'])
    initialize_row_dict['filing_id']=filing_id
    initialize_row_dict['url']=result['url']
    initialize_row_dict['filing_year']=int(result['filing_year'])
    initialize_row_dict['filing_period']=result['filing_period']
    initialize_row_dict['lobbyist_posted_by_name']=result['posted_by_name']
    
    initialize_row_dict['lobbyist_registrant_id']=result['registrant']['id']
    initialize_row_dict['lobbyist_registrant_name']=result['registrant']['name']
    initialize_row_dict['lobbyist_registrant_contact']=result['registrant']['contact_name']
    initialize_row_dict['client_id']=result['client']['id']
    initialize_row_dict['client_client_id']=result['client']['client_id']
    initialize_row_dict['client_name']=result['client']['name']
    initialize_row_dict['affiliated_organizations_present']=False
    if len(result['affiliated_organizations'])>0:
        initialize_row_dict['affiliated_organizations_present']=True
    initialize_row_dict['convictions_present']=False
    if len(result['conviction_disclosures'])>0:
        initialize_row_dict['convictions_present']=True
    return initialize_row_dict

In [388]:
def parse_lobbyists(df, lobbyists,details):

     lobby_dict={}
     lobby_dict['firm_name']=details['lobbyist_registrant_name']
     lobby_dict['client_name']=details['client_name']
     lobby_dict['general_issue_code']=details['general_issue_code']
     lobby_dict['description']=details['description']
     lobby_dict['filing_period']=details['filing_period']
     lobby_dict['filing_year']=details['filing_year']
     lobby_dict['url']=details['url']
     lobby_dict['filing_id']=details['filing_id']

     lobby_list=[]
     # unpack lobbyists list
     for lobbyist in lobbyists:
          lobby_dict['name']=lobbyist['lobbyist']['last_name']+', '+lobbyist['lobbyist']['first_name']
          lobby_dict['covered_position']='None'
          if 'covered_position' in lobbyist:
               lobby_dict['covered_position']=lobbyist['covered_position']
          lobby_dict['id']=lobbyist['lobbyist']['id']
          lobby_list.append(lobby_dict.copy())

     if df is None:
          df=pd.DataFrame(columns=['name','firm_name','client_name','general_issue_code','description','covered_position','filing_year','filing_period','url'])

     df=pd.concat([df,pd.DataFrame(lobby_list)])
     return df

In [395]:
query_filings=f'https://lda.senate.gov/api/v1/filings/?client_name="Exxon"'
        
f=requests.get(query_filings)

In [400]:
f.json()['results'][24]

{'url': 'https://lda.senate.gov/api/v1/filings/98e92e51-d223-4eaf-8633-d5089469f92e/',
 'filing_uuid': '98e92e51-d223-4eaf-8633-d5089469f92e',
 'filing_type': 'MM',
 'filing_type_display': 'Mid-Year Report',
 'filing_year': 2000,
 'filing_period': 'mid_year',
 'filing_period_display': 'Mid-Year (Jan 1 - Jun 30)',
 'filing_document_url': 'https://lda.senate.gov/filings/public/filing/98e92e51-d223-4eaf-8633-d5089469f92e/print/',
 'filing_document_content_type': 'application/pdf',
 'income': '12500.00',
 'expenses': None,
 'expenses_method': None,
 'expenses_method_display': None,
 'posted_by_name': None,
 'dt_posted': '2000-08-14T00:00:00-04:00',
 'termination_date': None,
 'registrant_country': 'United States of America',
 'registrant_ppb_country': 'United States of America',
 'registrant_address_1': None,
 'registrant_address_2': None,
 'registrant_different_address': None,
 'registrant_city': None,
 'registrant_state': None,
 'registrant_zip': None,
 'registrant': {'id': 29846,
  'url

In [403]:
# get govt entity names
entity_df=pd.DataFrame(government_entities)

corporations=['Exxon','ExxonMobil'] # 'exxon' alone includes ExxonMobil Exxon Mobil Corp, Exxon Mobil Corporation, etc. ['Exxon Mobil'.upper(),'ExxonMobil'.upper(),'Exxon'

row_list=[]  #initialize holder for each row (which corresponds to a single lobbying activity)
max_page=20

filing_id=0
lobbyists_df=None
for corporation in corporations:
    for year in list(range(1999,2025)):
        page=1
        while (page<max_page): 
            print(f'Querying page {page} for year {year}')
            query_filings=f'https://lda.senate.gov/api/v1/filings/?client_name="{corporation}"&filing_year={year}&page={page}'
            
            f=requests.get(query_filings)
            
            if 'detail' in f.json():
                break
            
            results=f.json()['results']
            page+=1  # increase the page for the next query
            print(f'{len(results)} filings found')
            if len(results)<25:
                page=max_page
            
            for result in results:
                row_dict_base=initialize_row(entity_df,result,filing_id)
                activities=result['lobbying_activities']
                
                for activity_count,activity in enumerate(activities):
                    row_dict=row_dict_base.copy()
                        #set up row dictionary using entity booleans
                    row_dict['activity_count']=activity_count
                    row_dict['general_issue_code']=activity['general_issue_code']
                    row_dict['description']=activity['description']
                    lobbyists_df=parse_lobbyists(lobbyists_df,activity['lobbyists'],row_dict)

                    lobbyist_id_list=[]
                    for lobbyist in activity['lobbyists']:
                        lobbyist_id_list.append(lobbyist['lobbyist']['id'])

                    row_dict['lobbyist_ids']='; '.join(['None' if x is None else str(x) for x in lobbyist_id_list])
                    for entity in activity['government_entities']:
                        row_dict[entity['name'].lower()]=1
                        
                    row_list.append(row_dict.copy())

                    row_dict.clear()
                filing_id+=1

tmp=pd.DataFrame(row_list)
entities_influenced=tmp[[x.lower() for x in list(entity_df['name'])]].sum()
zeroed=list(entities_influenced[entities_influenced == 0].index)
xom_df=tmp[[x for x in tmp.columns.values if x not in zeroed]]
xom_df
        

Querying page 1 for year 1999
20 filings found
Querying page 1 for year 2000
20 filings found
Querying page 1 for year 2001
17 filings found
Querying page 1 for year 2002
16 filings found
Querying page 1 for year 2003
11 filings found
Querying page 1 for year 2004
15 filings found
Querying page 1 for year 2005
22 filings found
Querying page 1 for year 2006
25 filings found
Querying page 2 for year 2006
5 filings found
Querying page 1 for year 2007
25 filings found
Querying page 2 for year 2007
13 filings found
Querying page 1 for year 2008
25 filings found
Querying page 2 for year 2008
25 filings found
Querying page 3 for year 2008
6 filings found
Querying page 1 for year 2009
25 filings found
Querying page 2 for year 2009
23 filings found
Querying page 1 for year 2010
25 filings found
Querying page 2 for year 2010
23 filings found
Querying page 1 for year 2011
25 filings found
Querying page 2 for year 2011
25 filings found
Querying page 3 for year 2011
7 filings found
Querying page 1 

,administration for children & families (acf),"agriculture, dept of (usda)","army, dept of (corps of engineers)","army, dept of (other)",bureau of land management (blm),"commerce, dept of (doc)",congressional budget office (cbo),consumer product safety commission (cpsc),council of economic advisers (cea),council on environmental quality (ceq),...,lobbyist_registrant_contact,client_id,client_client_id,client_name,affiliated_organizations_present,convictions_present,activity_count,general_issue_code,description,lobbyist_ids
0,0,0,0,0,0,0,0,0,0,0,...,DEBORAH WILLIAMS,116326,140,EXXON VALDEZ OIL SPILL LITIGTION PLAINTIFFS,False,False,0,TAX,None,36282
1,0,0,0,0,0,0,0,0,0,0,...,DEBORAH WILLIAMS,116326,140,EXXON VALDEZ OIL SPILL LITIGTION PLAINTIFFS,False,False,1,MAR,None,36282
2,0,0,0,0,0,0,0,0,0,0,...,DEBORAH WILLIAMS,116326,140,EXXON VALDEZ OIL SPILL LITIGTION PLAINTIFFS,False,False,2,ENV,None,36282
3,0,0,0,0,0,0,0,0,0,0,...,DEBORAH WILLIAMS,116326,140,EXXON VALDEZ OIL SPILL LITIGTION PLAINTIFFS,False,False,3,FUE,None,36282
4,0,0,0,0,0,0,0,0,0,0,...,DEBORAH WILLIAMS,116326,140,EXXON VALDEZ OIL SPILL LITIGTION PLAINTIFFS,False,False,4,LAW,None,36282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4382,0,0,0,0,0,0,0,0,0,0,...,ROBERT A. RUSSELL,198267,113,EXXONMOBIL CORPORATION,False,False,0,TAX,Discussions related to tax policy. Monitoring ...,59987; 59989
4383,0,0,0,0,0,0,0,0,0,0,...,ROBERT A. RUSSELL,198267,113,EXXONMOBIL CORPORATION,False,False,1,ENG,Discussions related to energy policy in the gl...,59987; 59989
4384,0,0,0,0,0,0,0,0,0,0,...,ROBERT A. RUSSELL,198267,113,EXXONMOBIL CORPORATION,False,False,0,TAX,Discussions related to tax policy. Monitoring ...,59987; 59989
4385,0,0,0,0,0,0,0,0,0,0,...,ROBERT A. RUSSELL,198267,113,EXXONMOBIL CORPORATION,False,False,1,ENG,Discussions related to energy policy in the gl...,59987; 59989


In [407]:
lobbyists_df

,name,firm_name,client_name,general_issue_code,description,covered_position,filing_year,filing_period,url,filing_id,id
0,"GANNETT, CRAIG",DAVIS WRIGHT TREMAINE LLP,EXXON VALDEZ OIL SPILL LITIGTION PLAINTIFFS,TAX,None,N/A,1999,mid_year,https://lda.senate.gov/api/v1/filings/38488650...,0.0,36282.0
0,"GANNETT, CRAIG",DAVIS WRIGHT TREMAINE LLP,EXXON VALDEZ OIL SPILL LITIGTION PLAINTIFFS,MAR,None,N/A,1999,mid_year,https://lda.senate.gov/api/v1/filings/38488650...,0.0,36282.0
0,"GANNETT, CRAIG",DAVIS WRIGHT TREMAINE LLP,EXXON VALDEZ OIL SPILL LITIGTION PLAINTIFFS,ENV,None,N/A,1999,mid_year,https://lda.senate.gov/api/v1/filings/38488650...,0.0,36282.0
0,"GANNETT, CRAIG",DAVIS WRIGHT TREMAINE LLP,EXXON VALDEZ OIL SPILL LITIGTION PLAINTIFFS,FUE,None,N/A,1999,mid_year,https://lda.senate.gov/api/v1/filings/38488650...,0.0,36282.0
0,"GANNETT, CRAIG",DAVIS WRIGHT TREMAINE LLP,EXXON VALDEZ OIL SPILL LITIGTION PLAINTIFFS,LAW,None,N/A,1999,mid_year,https://lda.senate.gov/api/v1/filings/38488650...,0.0,36282.0
...,...,...,...,...,...,...,...,...,...,...,...
0,"GOODMAN, MICHAEL","CORNERSTONE GOVERNMENT AFFAIRS, INC.",EXXONMOBIL,ENG,Congressional outreach on low carbon policies.,"Chief of Staff House Member Office (Kind), Leg...",2024,second_quarter,https://lda.senate.gov/api/v1/filings/6c804e0f...,1421.0,70188.0
1,"NELSON, TYLER","CORNERSTONE GOVERNMENT AFFAIRS, INC.",EXXONMOBIL,ENG,Congressional outreach on low carbon policies.,"Chief of Staff House Member Office, Deputy Chi...",2024,second_quarter,https://lda.senate.gov/api/v1/filings/6c804e0f...,1421.0,64545.0
2,"VENUTO, SARAH","CORNERSTONE GOVERNMENT AFFAIRS, INC.",EXXONMOBIL,ENG,Congressional outreach on low carbon policies.,"Senior Advisor(Manchin); Counsel, Comm Energy ...",2024,second_quarter,https://lda.senate.gov/api/v1/filings/6c804e0f...,1421.0,141958.0
3,"HODGSON, CHRISTOPHER","CORNERSTONE GOVERNMENT AFFAIRS, INC.",EXXONMOBIL,ENG,Congressional outreach on low carbon policies.,"Legislative Assistant (Scalise), Floor Assista...",2024,second_quarter,https://lda.senate.gov/api/v1/filings/6c804e0f...,1421.0,56105.0


In [408]:
xom_df.loc[xom_df.client_name=='HUNTON ANDREWS KURTH LLP (FORMERLY HUNTON & WILLIAMS LLP BEHALF OF EXXON MOBIL)']

,administration for children & families (acf),"agriculture, dept of (usda)","army, dept of (corps of engineers)","army, dept of (other)",bureau of land management (blm),"commerce, dept of (doc)",congressional budget office (cbo),consumer product safety commission (cpsc),council of economic advisers (cea),council on environmental quality (ceq),...,lobbyist_registrant_contact,client_id,client_client_id,client_name,affiliated_organizations_present,convictions_present,activity_count,general_issue_code,description,lobbyist_ids
2292,0,0,0,0,0,0,0,0,0,0,...,TERESA GORMAN,130049,113,HUNTON ANDREWS KURTH LLP (FORMERLY HUNTON & WI...,True,False,0,GOV,Clean Air Act Regulatory Issues\nClean Water A...,12923
2293,0,0,0,0,0,0,0,0,0,0,...,TERESA GORMAN,130049,113,HUNTON ANDREWS KURTH LLP (FORMERLY HUNTON & WI...,True,False,1,CAW,Clean Air Act Regulatory Issues\nClean Water A...,12923
2338,0,0,0,0,0,0,0,0,0,0,...,TERESA GORMAN,130049,113,HUNTON ANDREWS KURTH LLP (FORMERLY HUNTON & WI...,False,False,0,GOV,Risk Assessment and Regulatory Reform \nBenefi...,12923
2339,0,0,0,0,0,0,0,0,0,0,...,TERESA GORMAN,130049,113,HUNTON ANDREWS KURTH LLP (FORMERLY HUNTON & WI...,False,False,1,CAW,Clean Air Act Regulatory Issues\nClean Water A...,12923
2366,0,0,0,0,0,0,0,0,0,0,...,TERESA GORMAN,130049,113,HUNTON ANDREWS KURTH LLP (FORMERLY HUNTON & WI...,False,False,0,GOV,Risk Assessment and Regulatory Reform \nBenefi...,12923
2367,0,0,0,0,0,0,0,0,0,0,...,TERESA GORMAN,130049,113,HUNTON ANDREWS KURTH LLP (FORMERLY HUNTON & WI...,False,False,1,CAW,Clean Air Act Regulatory Issues\nClean Water A...,12923
2401,0,0,0,0,0,0,0,0,0,0,...,TERESA GORMAN,130049,113,HUNTON ANDREWS KURTH LLP (FORMERLY HUNTON & WI...,False,False,0,GOV,Risk Assessment and Regulatory Reform \nBenefi...,12923
2402,0,0,0,0,0,0,0,0,0,0,...,TERESA GORMAN,130049,113,HUNTON ANDREWS KURTH LLP (FORMERLY HUNTON & WI...,False,False,1,CAW,Clean Air Act Regulatory Issues\nBureau of Lan...,12923
2445,0,0,0,0,0,0,0,0,0,0,...,TERESA GORMAN,130049,113,HUNTON ANDREWS KURTH LLP (FORMERLY HUNTON & WI...,False,False,0,GOV,Risk Assessment and Regulatory Reform\nBenefit...,12923
2446,0,0,0,0,0,0,0,0,0,0,...,TERESA GORMAN,130049,113,HUNTON ANDREWS KURTH LLP (FORMERLY HUNTON & WI...,False,False,1,CAW,"H.R. 1430, the Honest and Open New EPA Science...",12923


In [415]:
xom_df.loc[xom_df.client_client_id==113]

,administration for children & families (acf),"agriculture, dept of (usda)","army, dept of (corps of engineers)","army, dept of (other)",bureau of land management (blm),"commerce, dept of (doc)",congressional budget office (cbo),consumer product safety commission (cpsc),council of economic advisers (cea),council on environmental quality (ceq),...,lobbyist_registrant_contact,client_id,client_client_id,client_name,affiliated_organizations_present,convictions_present,activity_count,general_issue_code,description,lobbyist_ids
1854,0,0,0,0,0,0,0,0,0,0,...,ROBERT A. RUSSELL,198267,113,EXXONMOBIL CORPORATION,False,False,0,ENG,"Issues related to energy policy, tax policy an...",59987; 59989
1855,0,0,0,0,0,0,0,0,0,0,...,ROBERT A. RUSSELL,198267,113,EXXONMOBIL CORPORATION,False,False,1,TRD,"Issues related to energy policy, tax policy an...",59987; 59989
1856,0,0,0,0,0,0,0,0,0,0,...,ROBERT A. RUSSELL,198267,113,EXXONMOBIL CORPORATION,False,False,2,TAX,"Issues related to energy policy, tax policy an...",59987; 59989
1868,0,0,0,0,0,0,0,0,0,0,...,ROBERT A. RUSSELL,198267,113,EXXONMOBIL CORPORATION,False,False,0,ENG,Issues related to energy policy.,59987; 59989
1869,0,0,0,0,0,0,0,0,0,0,...,ROBERT A. RUSSELL,198267,113,EXXONMOBIL CORPORATION,False,False,1,TRD,Issues related to international trade; the Rus...,59987; 59989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4377,0,0,0,0,0,0,0,0,0,0,...,ROBERT A. RUSSELL,198267,113,EXXONMOBIL CORPORATION,False,False,1,ENG,Discussions related to energy policy in the gl...,59987; 59989
4382,0,0,0,0,0,0,0,0,0,0,...,ROBERT A. RUSSELL,198267,113,EXXONMOBIL CORPORATION,False,False,0,TAX,Discussions related to tax policy. Monitoring ...,59987; 59989
4383,0,0,0,0,0,0,0,0,0,0,...,ROBERT A. RUSSELL,198267,113,EXXONMOBIL CORPORATION,False,False,1,ENG,Discussions related to energy policy in the gl...,59987; 59989
4384,0,0,0,0,0,0,0,0,0,0,...,ROBERT A. RUSSELL,198267,113,EXXONMOBIL CORPORATION,False,False,0,TAX,Discussions related to tax policy. Monitoring ...,59987; 59989


In [419]:

company_aliases=['EXXON MOBIL CORP',
       'EXXONMOBIL CHEMICAL CO', 'EXXON USA', 'EXXON CO USA',
       'EXXON MOBIL CORPORATION', 'EXXON MOBIL PRODUCTION CO',
       'EXXON MOBIL', 'EXXONMOBIL CORPORATION', 'EXXONMOBIL CORP',
       'EXXON MOBILE', 'EXXONMOBIL GLOBAL SERVICES COMPANY',
       'DCI GROUP, LLC, ON BEHALF OF EXXONMOBIL', 'EXXONMOBIL',
       'HUNTON ANDREWS KURTH LLP (FORMERLY HUNTON & WILLIAMS LLP BEHALF OF EXXON MOBIL)',
       'EXXON MOBIL COPORATION', 'EXXON MOBIL CORP.']
xom_df=xom_df.loc[xom_df.client_name.isin(company_aliases)]

lobbyists_df=lobbyists_df.loc[lobbyists_df.client_name.isin(company_aliases)]

In [342]:
xom_df[['dollars_spent_lobbying','filing_id']].groupby('filing_id').first()['dollars_spent_lobbying'].sum()

7260000.0

In [384]:
xom_df.to_csv('xom_lda_filings.csv')
lobbyists_df.to_csv('xom_lobbyists.csv')

In [443]:
xom_df.columns.values

array(['administration for children & families (acf)',
       'agriculture, dept of (usda)',
       'army, dept of (corps of engineers)', 'army, dept of (other)',
       'bureau of land management (blm)', 'commerce, dept of (doc)',
       'congressional budget office (cbo)',
       'consumer product safety commission (cpsc)',
       'council of economic advisers (cea)',
       'council on environmental quality (ceq)', 'defense, dept of (dod)',
       'education, dept of', 'energy, dept of',
       'environmental protection agency (epa)',
       'executive office of the president (eop)',
       'federal aviation administration (faa)',
       'federal bureau of investigation (fbi)',
       'federal energy regulatory commission (ferc)',
       'federal trade commission (ftc)',
       'food & drug administration (fda)',
       'general services administration (gsa)',
       'government accountability office (gao)',
       'health & human services, dept of (hhs)',
       'homeland security,

In [450]:
tmp=xom_df[['dollars_spent_lobbying','lobbyist_registrant_name','filing_id']].groupby(['lobbyist_registrant_name','filing_id']).first().reset_index()
total_dollars_spent_df=tmp.groupby('lobbyist_registrant_name').agg({'dollars_spent_lobbying':'sum','filing_id':'count'}).sort_values(by='dollars_spent_lobbying',ascending=False)
total_dollars_spent_df

,dollars_spent_lobbying,filing_id
lobbyist_registrant_name,,
EXXON MOBIL CORP,304096942.0,88
ERNST & YOUNG LLP (WASHINGTON COUNCIL ERNST & YOUNG),4240000.0,126
"DCI GROUP, L.L.C.",3930000.0,53
KELLEY DRYE & WARREN LLP,3360000.0,69
"THE NICKLES GROUP, LLC",3254500.0,54
"THE SIMMONS & RUSSELL GROUP, LLC",2480000.0,84
"ML STRATEGIES, LLC",2355000.0,45
MICHAEL SOLON-CAPITOL LEGISTICS,2235200.0,62
"CLOUD & COMPANY, LLC",2060000.0,52


In [458]:
xom_dci_lobbyists=list(lobbyists_df.loc[lobbyists_df.firm_name=='DCI GROUP, L.L.C.'].name.unique())

In [426]:
counts_df=lobbyists_df[['name','firm_name']].groupby('firm_name').count().sort_values(by='name')
counts_df.columns=['firm_total_filings']
df=lobbyists_df.merge(counts_df,left_on='firm_name',right_index=True)
counts_df=lobbyists_df[['name','firm_name']].groupby('name').count().sort_values(by='firm_name')
counts_df.columns=['total_number_of_filings']
df=df.merge(counts_df,left_on='name',right_index=True)


,name
firm_name,
WILLIAM HILLEARY,1
"THOMAS ADVISORS, INC",4
"OMB PROFESSIONALS, INC.",6
SOLUTIONS CONSULTING,7
"VALIS ASSOCIATES, LLC",8
"CASSIDY & ASSOCIATES, INC.",9
KERRY CAMMACK,10
CLARK & WEINSTOCK,10
MILLER & CHEVALIER CHARTERED,14


In [431]:
lobbyists_df[['name','firm_name','client_name','total_number_of_filings']].groupby('name').first().sort_values(by='total_number_of_filings',ascending=False).head(50)

,firm_name,client_name,total_number_of_filings
name,,,
"MITCHELL, JEANNE",EXXON MOBIL CORP,EXXON MOBIL CORP,824
"RUSSELL, ROBERT",THE INGRAM GROUP LLC,EXXONMOBIL CORPORATION,390
"SIMMONS, KYLE",THE INGRAM GROUP LLC,EXXONMOBIL CORPORATION,390
"HAINES, R",EXXON MOBIL CORP,EXXON MOBIL CORP,306
"KARDON, JOSH",CAPITOL HILL CONSULTING GROUP,EXXONMOBIL,290
"BUCHHOLTZ, W",EXXON MOBIL CORP,EXXON MOBIL CORP,288
"HAMMER, A",EXXON MOBIL CORP,EXXON MOBIL CORP,283
"WOOD, DANA",KELLEY DRYE & WARREN LLP,EXXON MOBIL CORPORATION,271
"FARIELLO, THERESA",EXXON MOBIL CORP,EXXON MOBIL CORP,257


In [459]:
lob1=requests.get('https://lda.senate.gov/api/v1/lobbyists/?registrant_name="DCI GROUP, L.L.C."&page=1')
lob2=requests.get('https://lda.senate.gov/api/v1/lobbyists/?registrant_name="DCI GROUP, L.L.C."&page=2')
dci_lobbyists=lob1.json()['results']+lob2.json()['results']


In [462]:
all_dci_lobbyists=[]
for dci in dci_lobbyists:
    all_dci_lobbyists.append(dci['last_name']+', '+dci['first_name'])

sorted(all_dci_lobbyists)

['ARCENEAUX, ALYSSA',
 'BASYE, GEOFFREY',
 'BENTZEL, CARL',
 'BIRKHOFER, MATT',
 'BORLAND, KEVIN',
 'CAOL, DREW',
 'CHIANCHETTE, PETER',
 'CLINGHAM, GAVIN',
 'COLE, DREW',
 'CRADDOCK, FRANK',
 'DAVENPORT, DOUG',
 'ENYON, TEDDY',
 'FEATHER, TONY',
 'FLOOD, ANGELA',
 'FRICKS, JOSEPH',
 'FRITZ, TRISH',
 'GRILLO, RYAN',
 'HORRELL, CHAD',
 'KENNEDY, BRIAN',
 'LACIVITA, CHRIS',
 'LARSON, CHRIS',
 'LAWRENCE, LINDSAY',
 'LEGER, STEPHANIE',
 'MATRICARDI, ED',
 'MCCABE, BRIAN',
 'MCCULLOUGH, CATHERINE',
 'MCKENNA, ANDREW',
 'OLSON, GRANT',
 'OSCHAL, JENNIFER',
 'PEARCE, DAVID',
 'PETERSON, JUSTIN',
 'RICHARDSON, CRAIG',
 'RYAN, PAUL',
 'SHOCKEY, ALEX',
 'STRATTON, HAL',
 'SYNHORST, TOM',
 'TOBIN, JAMES',
 'TOOZE, COLIN',
 'TURNER, JONA',
 'VIDENIEKS, LINDSAY',
 'WHITAKER, STEPHEN',
 'WILKINSON, SCOTT',
 'WOLF, OLIVER',
 'ZIEBART, GEOFFREY']

In [ ]:
xom_df.loc[xom_df.f]

In [463]:
xom_dci_lobbyists

['BIRKHOFER, MATT',
 'BENTZEL, CARL',
 'COLE, DREW',
 'CRADDOCK, FRANK',
 'PEARCE, DAVID',
 'FLOOD, ANGELA']

In [129]:
xom_dict={}
for x in xom:
    xom_dict['year']=int(x['filing_year'])
    xom_dict['rest']=

{'url': 'https://lda.senate.gov/api/v1/filings/ae63a422-c47c-4ed0-9099-fa11613cbd6d/',
 'filing_uuid': 'ae63a422-c47c-4ed0-9099-fa11613cbd6d',
 'filing_type': 'YY',
 'filing_type_display': 'Year-End Report',
 'filing_year': 2000,
 'filing_period': 'year_end',
 'filing_period_display': 'Year-End (July 1 - Dec 31)',
 'filing_document_url': 'https://lda.senate.gov/filings/public/filing/ae63a422-c47c-4ed0-9099-fa11613cbd6d/print/',
 'filing_document_content_type': 'application/pdf',
 'income': '0.00',
 'expenses': None,
 'expenses_method': None,
 'expenses_method_display': None,
 'posted_by_name': None,
 'dt_posted': '1940-02-21T00:00:00-05:00',
 'termination_date': None,
 'registrant_country': 'United States of America',
 'registrant_ppb_country': 'United States of America',
 'registrant_address_1': None,
 'registrant_address_2': None,
 'registrant_different_address': None,
 'registrant_city': None,
 'registrant_state': None,
 'registrant_zip': None,
 'registrant': {'id': 39837,
  'url': 

In [433]:
lob=requests.get('https://lda.senate.gov/api/v1/lobbyists/')
filing_types=requests.get('https://lda.senate.gov/api/v1/constants/filing/filingtypes/')
general_issues=requests.get('https://lda.senate.gov/api/v1/constants/filing/lobbyingactivityissues/')
govt_entities=requests.get('https://lda.senate.gov/api/v1/constants/filing/governmententities/')
countries=requests.get('https://lda.senate.gov/api/v1/constants/general/countries/')
states=requests.get('https://lda.senate.gov/api/v1/constants/general/states/')
prefixes=requests.get('https://lda.senate.gov/api/v1/constants/lobbyist/prefixes/')
suffixes=requests.get('https://lda.senate.gov/api/v1/constants/lobbyist/suffixes/')
contribution_items=requests.get('https://lda.senate.gov/api/v1/constants/contribution/itemtypes/')



In [453]:
lob=requests.get('https://lda.senate.gov/api/v1/lobbyists/?registrant_name="DCI GROUP, L.L.C."&page=2')
lob.json()['results']

[{'id': 35116,
  'prefix': None,
  'prefix_display': None,
  'first_name': 'BRIAN',
  'nickname': None,
  'middle_name': None,
  'last_name': 'KENNEDY',
  'suffix': None,
  'suffix_display': None,
  'registrant': {'id': 61174,
   'url': 'https://lda.senate.gov/api/v1/registrants/61174/',
   'house_registrant_id': 36371,
   'name': 'DCI GROUP, L.L.C.',
   'description': 'Public affairs',
   'address_1': '2000 K Street, NW',
   'address_2': '9th Floor',
   'address_3': None,
   'address_4': None,
   'city': 'Washington',
   'state': 'DC',
   'state_display': 'District of Columbia',
   'zip': '20006',
   'country': 'US',
   'country_display': 'United States of America',
   'ppb_country': 'US',
   'ppb_country_display': 'United States of America',
   'contact_name': 'REBECCA NEVILLE',
   'contact_telephone': '+1 202-777-1566',
   'dt_updated': '2024-07-25T09:38:49.084179-04:00'}},
 {'id': 35673,
  'prefix': None,
  'prefix_display': None,
  'first_name': 'KEVIN',
  'nickname': None,
  'mid

In [484]:
contributions=[]
for page in range(1,4):
    c=requests.get(f'https://lda.senate.gov/api/v1/contributions/?registrant_name="DCI GROUP, L.L.C."&filing_type=YY&page={page}')
    contributions=contributions+c.json()['results']


In [494]:
row_list=[]

for dci in contributions:
    row_dict={}
    for i in dci['contribution_items']:
        row_dict['contribution_type']=i['contribution_type']
        row_dict['contributor_name']=i['contributor_name']
        row_dict['payee_name']=i['payee_name']
        row_dict['honoree_name']=i['honoree_name']
        row_dict['usd']=float(i['amount'])
        row_dict['date']=i['date']
        row_dict['filing_year']=dci['filing_year']
        row_dict['registrant_name']=dci['registrant']['name']
        row_list.append(row_dict.copy())
dci_contributions_df=pd.DataFrame(row_list)

In [532]:
def strip_terms(x,terms):
    for term in terms:
        x=x.replace(term,'')
    
    return x.split(' (')[0].title()

In [535]:
terms=['Senator ','US ','Rep. ','U.S. ','Representative ','Sen. ','Congressional ','Candidate ','Congressman ','for ','Friends of ']

print(dci_contributions_df.honoree_name.nunique())
print(dci_contributions_df.stripped_honoree_name.nunique())


281
181


In [534]:
dci_contributions_df.honoree_name.unique()
terms=['Senator ','US ','Rep. ','Congresswoman ','U.S. ',' 08',' Jr.','Jr','Resident Commissioner of Puerto Rico ','Senate ','Campaign','Presidential ','candidate ','Candidate ','Representative ','Sen. ','House ','Congressional ','Candidate ','Representative-Elect ','Campaign Committee','Committee','Congressman ','for ','Friends of ',' Senate',' Congress','Friend of ','- In-Kind Donation']
dci_contributions_df['stripped_honoree_name']=[strip_terms(x,terms) for x in dci_contributions_df.honoree_name]
dci_contributions_df.stripped_honoree_name.unique()


array(['Paul Carmouche', 'Mark Pryor', 'Parker Griffith', 'Alan Mollohan',
       'Democratic  ', 'Gordon Smith', 'Mike Johanns', 'Mitch Mcconnell',
       'Lamar Alexander', 'John Salazar', 'Peter Defazio', 'Mark Souder',
       'The Freedom Project', 'Mccain ', 'John Boehner', 'Mcconnell ',
       'Dci Group Pac', 'Ann Barth', 'Souder', 'Allan Mollohan',
       'Alexander', 'Defazio', 'Johanns', 'Gordan Smith',
       'Kirsten Gillibrand', 'Charlie Melancon', 'Jim Oberstar',
       'Portman', 'Gillibrand', 'Castillo', 'Jim Clyburn',
       'Kendrick Meek Florida', 'Georgians Isakson', 'Charlie Melancon  ',
       'Boyd', 'Pete King', 'James Oberstar', 'Kendrick Meek',
       'Allen Boyd', 'Kirsten Gillibrand ', 'David Castillo',
       'James Clyburn', 'Johnny Isakson', 'Rob Portman', 'Peter King',
       'Chad Causey', 'Nrcc', 'Kelly Ayotte', 'Bill Flores',
       'Mary Landrieu', 'N/A', 'Connie Mack', 'Roy Blunt', 'Eric Cantor',
       'John Dingell', 'Chris Coons', 'Karen Diebel',

In [570]:
yy=list(dci_contributions_df.stripped_honoree_name.unique())
dci_contributions_df['stripped_honoree_name']=[x.strip() for x in dci_contributions_df['stripped_honoree_name']]
dci_contributions_df['stripped_honoree_name_sim']=dci_contributions_df['stripped_honoree_name']
for y in yy:
    dci_contributions_df['stripped_honoree_name_sim']=[y if x in y else x for x in dci_contributions_df.stripped_honoree_name_sim]
yy=list(dci_contributions_df.stripped_honoree_name_sim.unique())
for y in yy:
    dci_contributions_df['stripped_honoree_name_sim']=[y if x in y else x for x in dci_contributions_df.stripped_honoree_name_sim]

In [573]:
yy=list(dci_contributions_df.stripped_honoree_name_sim.unique())
for term, replacement in {'Allan':'Alan','Jim':'James','Gordan':'Gordon','&':'And','Georgians':'Johnny','M.':'Mary','Pete':'Peter'}.items():
    dci_contributions_df['stripped_honoree_name_sim']=[x.replace(term,replacement) for x in dci_contributions_df.stripped_honoree_name_sim]

In [578]:
dci_contributions_df[['stripped_honoree_name_sim','usd']].groupby('stripped_honoree_name_sim').sum().sort_values(by='usd',ascending=False).sum()

usd    421845.57
dtype: float64

In [556]:
import pandas as pd
from name_matching.name_matcher import NameMatcher

# define a dataset with bank names
df_a = pd.DataFrame({'recipient1':y})

# alter each of the bank names a bit to test the matching
df_b = pd.DataFrame({'recipient2':sorted(y)})

# initialise the name matcher
matcher = NameMatcher(number_of_matches=5, 
                      legal_suffixes=True, 
                      common_words=False, 
                      top_n=50, 
                      verbose=True)

# adjust the distance metrics to use
matcher.set_distance_metrics(['bag', 'typo', 'refined_soundex'])

# load the data to which the names should be matched
matcher.load_and_process_master_data(column='recipient1',
                                     df_matching_data=df_a, 
                                     transform=True)

# perform the name matching on the data you want matched
matches = matcher.match_names(to_be_matched=df_b, 
                              column_matching='recipient2')

# combine the datasets based on the matches
#combined = pd.merge(df_a, matches, how='left', left_index=True, right_on='match_index')
#combined#combined = pd.merge(combined, df_b, how='left', left_index=True, right_index=True)

preprocessing...

preprocessing complete 
 searching for matches...



100%|██████████| 1/1 [00:00<00:00, 451.19it/s]


possible matches found   
 fuzzy matching...



100%|██████████| 181/181 [00:04<00:00, 44.35it/s]

done


In [557]:
#matches.drop(['match_name_0','score_0'],axis=1,inplace=True)
matches

,original_name,match_name_0,score_0,match_index_0,match_name_1,score_1,match_index_1,match_name_2,score_2,match_index_2,match_name_3,score_3,match_index_3,match_name_4,score_4,match_index_4
0,adrian smith,adrian smith,100.0,178,gordan smith,66.269841,23,gordon smith,62.103175,5,tim scott,46.250100,163,cedric richmond,45.311463,114
1,alan mollohan,alan mollohan,100.0,3,allan mollohan,95.238095,19,allen boyd,46.620426,38,salmon,44.056323,123,charlie melancon,42.293390,25
2,alexander,alexander,100.0,20,lamar alexander,60.833333,8,landrieu,55.555556,68,m landrieu,50.841510,92,altmire,47.381086,88
3,allan mollohan,allan mollohan,100.0,19,alan mollohan,95.238095,3,allen boyd,49.480871,38,charlie melancon,42.903584,25,charlie melancon,42.903584,33
4,allen boyd,allen boyd,100.0,38,allan mollohan,49.480871,19,alan mollohan,46.620426,3,kelly ayotte,43.233145,47,alexander,39.595599,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,tom graves,tom graves,100.0,140,garret graves,55.165020,149,tom perez,53.916197,173,jim oberstar,42.934543,26,john barrasso,42.756049,73
177,tom perez,tom perez,100.0,173,tom graves,53.916197,140,jim oberstar,46.715879,26,matt mowers,40.238423,171,bob menendez,39.238322,96
178,tommy thompson,tommy thompson,100.0,99,jim matheson,49.923220,102,johnny isakson,42.082998,42,pete olson,41.173555,104,olympia snowe,39.281075,75
179,various,various,100.0,180,barasso,43.650794,83,mark souder,42.121212,11,chris coons,36.060606,55,marco rubio,34.299579,167


In [496]:
dci_contributions_df[['filing_year','usd']].groupby('filing_year').sum()

,usd
filing_year,
2008,73100.00
2009,34600.00
2010,49450.00
2011,46500.00
2012,28750.00
2013,61500.00
2014,24039.54
2015,52000.00
2016,19500.00


In [98]:
firms=[]
#lobbyists=lobbyists.json()['results']
for lobby in lobbyists:
    print(lobby)
    firms.append(lobby['registrant']['name'])

{'id': 1, 'prefix': None, 'prefix_display': None, 'first_name': 'ALICE', 'nickname': None, 'middle_name': None, 'last_name': 'DUCQ', 'suffix': None, 'suffix_display': None, 'registrant': {'id': 51968, 'url': 'https://lda.senate.gov/api/v1/registrants/51968/', 'house_registrant_id': 34851, 'name': 'MAYBERRY & ASSOCIATES, LLC', 'description': None, 'address_1': '131 EAST BROAD STREET, #206', 'address_2': None, 'address_3': None, 'address_4': None, 'city': 'FALLS CHURCH', 'state': 'VA', 'state_display': 'Virginia', 'zip': '22046', 'country': 'US', 'country_display': 'United States of America', 'ppb_country': 'US', 'ppb_country_display': 'United States of America', 'contact_name': 'MR. PETER MAYBERRY', 'contact_telephone': '+1 703-538-8804', 'dt_updated': '2022-04-06T16:07:54.085048-04:00'}}
{'id': 2, 'prefix': None, 'prefix_display': None, 'first_name': 'TOBIN', 'nickname': None, 'middle_name': 'L', 'last_name': 'SMITH', 'suffix': None, 'suffix_display': None, 'registrant': {'id': 23964, 

In [95]:
firms

['MAYBERRY & ASSOCIATES, LLC',
 'MASSACHUSETTS INSTITUTE OF TECHNOLOGY',
 'GREENLEE PARTNERS, LLC',
 'KSA CONSULTING',
 'NATIONAL FAIR HOUSING ALLIANCE',
 'WILSON ELSER MOSLOWITZ EDELMAN & DICKER, LLP',
 'AVOQ, LLC (FRA TEAM SUBJECT MATTER, LLC)',
 'GORDLEY ASSOCIATES',
 'MEAT INSTITUTE (FKA NORTH AMERICAN MEAT INSTITUTE)',
 'OPTICAL SOCIETY OF AMERICA',
 'DENNY MILLER ASSOCIATES',
 'AMERICAN FOREST & PAPER ASSOCIATION',
 'PEOPLE FOR THE AMERICAN WAY',
 'CONSUMER TECHNOLOGY ASSOCIATION',
 'MR. ROBERT WILSON',
 'HURT, NORTON & ASSOCIATES, INC.',
 'SOMMER BARNARD PC',
 'NATIONAL ASSOCIATION OF REALTORS',
 'FEMINIST MAJORITY, THE',
 'FRANKLEE SPEAKING LLC',
 'QUINN THOMAS PUBLIC AFFAIRS',
 'BKSH & ASSOCIATES',
 'NATIONAL ASSOCIATION OF HOME BUILDERS',
 'GEORGIA-PACIFIC LLC',
 'CHAMBER OF COMMERCE OF THE U.S.A.']

In [84]:
payee='Friends of Max Baucus'.replace(' ','%20')
year='2008'
contributor='Crossroads Strategies, LLC'.replace(' ','%20')
year='2023'
query_all_contributions=f'https://lda.senate.gov/api/v1/contributions/?contribution_payee={payee}&filing_year={year}'
c=requests.get(query_all_contributions)
baucus=c.json()['results']


In [90]:
contributor='Crossroads Strategies'.replace(' ','%20')
year='2023'
query_all_contributions=f'https://lda.senate.gov/api/v1/contributions/?contribution_contributor={contributor}&filing_year={year}'
c=requests.get(query_all_contributions)
crossroads=c.json()['results']

In [91]:
crossroads

[]

In [81]:
total=0

for cc in baucus:
    contributions=cc['contribution_items']
    for i in contributions:

        if i['payee_name'].lower()=='friends of max baucus':
            print(f'{i["contributor_name"]}, amount: ${i["amount"]}')
            total=total+float(i["amount"])

In [82]:
total

0

In [45]:
govt_entities_df=pd.DataFrame(govt_entities.json())
contribution_items_df=pd.DataFrame(contribution_items.json())
general_issues_df=pd.DataFrame(general_issues.json())
filing_types_df=pd.DataFrame(filing_types.json())

In [24]:
constants=requests.get('https://lda.senate.gov/api/v1/constants/')
constants.json()

JSONDecodeError: Expecting value: line 3 column 1 (char 2)

In [21]:
r.json()

{'filings': 'https://lda.senate.gov/api/v1/filings/',
 'contributions': 'https://lda.senate.gov/api/v1/contributions/',
 'registrants': 'https://lda.senate.gov/api/v1/registrants/',
 'clients': 'https://lda.senate.gov/api/v1/clients/',
 'lobbyists': 'https://lda.senate.gov/api/v1/lobbyists/',
 'constants/filing/filingtypes': 'https://lda.senate.gov/api/v1/constants/filing/filingtypes/',
 'constants/filing/lobbyingactivityissues': 'https://lda.senate.gov/api/v1/constants/filing/lobbyingactivityissues/',
 'constants/filing/governmententities': 'https://lda.senate.gov/api/v1/constants/filing/governmententities/',
 'constants/general/countries': 'https://lda.senate.gov/api/v1/constants/general/countries/',
 'constants/general/states': 'https://lda.senate.gov/api/v1/constants/general/states/',
 'constants/lobbyist/prefixes': 'https://lda.senate.gov/api/v1/constants/lobbyist/prefixes/',
 'constants/lobbyist/suffixes': 'https://lda.senate.gov/api/v1/constants/lobbyist/suffixes/',
 'constants/

In [17]:
rr=requests.get(r.json()['filings']+'?client_name=exxon%20mobil&filing_year=2023')

In [19]:
rr.json()

{'count': 34,
 'next': 'https://lda.senate.gov/api/v1/filings/?client_name=exxon+mobil&filing_year=2023&page=2',
 'previous': None,
 'results': [{'url': 'https://lda.senate.gov/api/v1/filings/17a0b97c-5028-489e-b0e3-38bd0f74cd92/',
   'filing_uuid': '17a0b97c-5028-489e-b0e3-38bd0f74cd92',
   'filing_type': 'Q1',
   'filing_type_display': '1st Quarter - Report',
   'filing_year': 2023,
   'filing_period': 'first_quarter',
   'filing_period_display': '1st Quarter (Jan 1 - Mar 31)',
   'filing_document_url': 'https://lda.senate.gov/filings/public/filing/17a0b97c-5028-489e-b0e3-38bd0f74cd92/print/',
   'filing_document_content_type': 'text/html',
   'income': '30000.00',
   'expenses': None,
   'expenses_method': None,
   'expenses_method_display': None,
   'posted_by_name': 'Timothy F. Hannegan',
   'dt_posted': '2023-04-18T07:36:50-04:00',
   'termination_date': None,
   'registrant_country': 'United States of America',
   'registrant_ppb_country': None,
   'registrant_address_1': '701 P

In [16]:
rr.json()['results'][0].keys()

dictionary=rr.json()['results'][0]
dictionary=flatten(dictionary)

for key,value in dictionary.items():
    print(f'{key}: {value}')

url: https://lda.senate.gov/api/v1/filings/7b9b34c8-31b1-4720-80e5-cc68f1d8c5db/
filing_uuid: 7b9b34c8-31b1-4720-80e5-cc68f1d8c5db
filing_type: Q1
filing_type_display: 1st Quarter - Report
filing_year: 2023
filing_period: first_quarter
filing_period_display: 1st Quarter (Jan 1 - Mar 31)
filing_document_url: https://lda.senate.gov/filings/public/filing/7b9b34c8-31b1-4720-80e5-cc68f1d8c5db/print/
filing_document_content_type: text/html
income: 20000.00
expenses: None
expenses_method: None
expenses_method_display: None
posted_by_name: Adina Mincher
dt_posted: 2023-04-13T09:49:37-04:00
termination_date: None
registrant_country: US
registrant_ppb_country: US
registrant_address_1: 300 Tingey Street SE
registrant_address_2: Suite 202
registrant_different_address: False
registrant_city: Washington
registrant_state: DC
registrant_zip: 20003
registrant_id: 70175
registrant_url: https://lda.senate.gov/api/v1/registrants/70175/
registrant_house_registrant_id: 30174
registrant_name: MERCURY PUBLIC 

In [10]:
flatten(dictionary)

{'url': 'https://lda.senate.gov/api/v1/filings/7b9b34c8-31b1-4720-80e5-cc68f1d8c5db/',
 'filing_uuid': '7b9b34c8-31b1-4720-80e5-cc68f1d8c5db',
 'filing_type': 'Q1',
 'filing_type_display': '1st Quarter - Report',
 'filing_year': 2023,
 'filing_period': 'first_quarter',
 'filing_period_display': '1st Quarter (Jan 1 - Mar 31)',
 'filing_document_url': 'https://lda.senate.gov/filings/public/filing/7b9b34c8-31b1-4720-80e5-cc68f1d8c5db/print/',
 'filing_document_content_type': 'text/html',
 'income': '20000.00',
 'expenses': None,
 'expenses_method': None,
 'expenses_method_display': None,
 'posted_by_name': 'Adina Mincher',
 'dt_posted': '2023-04-13T09:49:37-04:00',
 'termination_date': None,
 'registrant_country': 'US',
 'registrant_ppb_country': 'US',
 'registrant_address_1': '300 Tingey Street SE',
 'registrant_address_2': 'Suite 202',
 'registrant_different_address': False,
 'registrant_city': 'Washington',
 'registrant_state': 'DC',
 'registrant_zip': '20003',
 'registrant_id': 70175,

In [ ]:
query = f"https://developer.nrel.gov/api/pvwatts/v8.json?api_key={config['apikey']}&lat={lat}&lon={lon}&system_capacity={config['kw']}&azimuth={azimuth}&tilt={tilt}&array_type=1&module_type=1&losses=10"
                response = requests.get(query, timeout=20)
                info_dict["ac_annual"] = response.json()["outputs"]["ac_annual"]